In [1]:
import os
import shutil
import numpy as np
import glob   
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Input, Add,Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import  plot_model
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.applications import ResNet50


from IPython.display import SVG
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline


In [2]:
from tensorflow.keras.applications.imagenet_utils import preprocess_input

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Data Source

- Flower DataSet from Kaggle
- Source: https://www.kaggle.com/alxmamaev/flowers-recognition

In [4]:
base_dir = "./flowers"


In [5]:
#create two folders (train and validation)
#train_dir = os.path.join(base_dir, 'train')
#os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
#Under train folder create five folders 

train_daisy_dir = os.path.join(train_dir, 'daisy')
os.mkdir(train_daisy_dir)
train_dandelion_dir = os.path.join(train_dir, 'dandelion')
os.mkdir(train_dandelion_dir)
train_rose_dir = os.path.join(train_dir, 'rose')
os.mkdir(train_rose_dir)
train_sunflower_dir = os.path.join(train_dir, 'sunflower')
os.mkdir(train_sunflower_dir)
train_tulip_dir = os.path.join(train_dir, 'tulip')
os.mkdir(train_tulip_dir)
#Under validation folder create five folders 

   

validation_daisy_dir = os.path.join(validation_dir, 'daisy')
os.mkdir(validation_daisy_dir)
validation_dandelion_dir = os.path.join(validation_dir, 'dandelion')
os.mkdir(validation_dandelion_dir)
validation_rose_dir = os.path.join(validation_dir, 'rose')
os.mkdir(validation_rose_dir)
validation_sunflower_dir = os.path.join(validation_dir, 'sunflower')
os.mkdir(validation_sunflower_dir)
validation_tulip_dir = os.path.join(validation_dir, 'tulip')
os.mkdir(validation_tulip_dir)




In [6]:
def split_dataset_into_test_and_train_sets(all_data_dir=base_dir, 
                                           training_data_dir="./flowers/train", 
                                           testing_data_dir="./flowers/validation", 
                                           testing_data_pct=0.2):
    # Recreate testing and training directories
    if testing_data_dir.count('/') > 1:
        shutil.rmtree(testing_data_dir, ignore_errors=False)
        os.makedirs(testing_data_dir)
        print("Successfully cleaned directory " + testing_data_dir)
    else:
        print("Refusing to delete testing data directory " + testing_data_dir + " as we prevent you from doing stupid things!")

    if training_data_dir.count('/') > 1:
        shutil.rmtree(training_data_dir, ignore_errors=False)
        os.makedirs(training_data_dir)
        print("Successfully cleaned directory " + training_data_dir)
    else:
        print("Refusing to delete testing data directory " + training_data_dir + " as we prevent you from doing stupid things!")

    num_training_files = 0
    num_testing_files = 0

    for subdir, dirs, files in os.walk(all_data_dir):
        try: 
            category_name = os.path.basename(subdir)

            # Don't create a subdirectory for the root directory
            print(category_name + " vs " + os.path.basename(all_data_dir))
            if category_name == os.path.basename(all_data_dir):
                continue

            training_data_category_dir = training_data_dir + '/' + category_name
            testing_data_category_dir = testing_data_dir + '/' + category_name

            if not os.path.exists(training_data_category_dir):
                os.mkdir(training_data_category_dir)

            if not os.path.exists(testing_data_category_dir):
                os.mkdir(testing_data_category_dir)

            for file in files:
                input_file = os.path.join(subdir, file)
                if np.random.rand(1) < testing_data_pct:
                    shutil.copy(input_file, testing_data_dir + '/' + category_name + '/' + file)
                    num_testing_files += 1
                else:
                    shutil.copy(input_file, training_data_dir + '/' + category_name + '/' + file)
                    num_training_files += 1
        except:
            continue

    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")


In [7]:
split_dataset_into_test_and_train_sets()

Successfully cleaned directory ./flowers/validation
Successfully cleaned directory ./flowers/train
flowers vs flowers
daisy vs flowers
rose vs flowers
tulip vs flowers
train vs flowers
daisy vs flowers
rose vs flowers
tulip vs flowers
dandelion vs flowers
validation vs flowers
daisy vs flowers
rose vs flowers
tulip vs flowers
train vs flowers
dandelion vs flowers
sunflower vs flowers
Processed 3411 training files.
Processed 920 testing files.


In [8]:
import os

dir_path_1 = './flowers/train/train'
dir_path_2 = './flowers/train/validation'
dir_path_3 = './flowers/validation/train'
dir_path_4 = './flowers/validation/validation'

try:
    os.rmdir(dir_path_1)
    os.rmdir(dir_path_2)
    os.rmdir(dir_path_3)
    os.rmdir(dir_path_4)    
except OSError as e:
    print("Removed already")

In [9]:
# Number of classes in dataset
num_classes = 5

def get_model():
    with tf.device('/cpu:0'):
        # Get base model 
        # Here we are using ResNet50 as base model | Actual softmax layer we do not have to include.
        base_model = ResNet50(weights='imagenet', include_top=False)

        # As we are using ResNet model only for feature extraction and not adjusting the weights
        # we freeze the layers in base model
        for layer in base_model.layers:
            layer.trainable = False

        # Get base model output 
        base_model_ouput = base_model.output

        # Adding our own layer 
        x = GlobalAveragePooling2D()(base_model_ouput)
        # Adding fully connected layer
        x = Dense(512, activation='relu')(x)
        x = Dense(num_classes, activation='softmax', name='fcnew')(x)

        model = Model(inputs=base_model.input, outputs=x)
        return model

In [10]:
# Get the model
model = get_model()
# Compile it
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# Summary of model
model.summary()

Metal device set to: Apple M1


2022-01-01 15:50:27.031470: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-01 15:50:27.031576: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                           

In [11]:
training_dir = "./flowers/train"
testing_dir = "./flowers/validation"

In [12]:
def image_data_gen_func():
    with tf.device('/cpu:0'):
        global train_data_gen
        global valid_data_gen
        global train_generator
        global valid_generator
        train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
        valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
        train_generator = train_data_gen.flow_from_directory(training_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')
        valid_generator = valid_data_gen.flow_from_directory(testing_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')

In [13]:
# Defining the imagedatagenerator for train and test image for pre-processing
# We don't give horizonal_flip or other preprocessing for validation data generator

image_size = 224
batch_size = 64
image_data_gen_func()


Found 3408 images belonging to 5 classes.
Found 918 images belonging to 5 classes.


In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                           

In [15]:
# Training the fully conncected layer for initial epochs
epochs = 5

# Training the model

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

/var/folders/gw/057r0tcd6cg7wvd0kgbh499m0000gn/T/ipykernel_77395/1208860038.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(
2022-01-01 15:50:28.373297: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/5


2022-01-01 15:50:29.153645: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - ETA: 0s - loss: 0.7632 - accuracy: 0.7237

2022-01-01 15:51:05.399269: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - 47s 856ms/step - loss: 0.7632 - accuracy: 0.7237 - val_loss: 0.4794 - val_accuracy: 0.8438
Epoch 2/5
53/53 [==============================] - 46s 856ms/step - loss: 0.4190 - accuracy: 0.8583 - val_loss: 0.3936 - val_accuracy: 0.8672
Epoch 3/5
53/53 [==============================] - 45s 836ms/step - loss: 0.3466 - accuracy: 0.8867 - val_loss: 0.3591 - val_accuracy: 0.8717
Epoch 4/5
53/53 [==============================] - 45s 843ms/step - loss: 0.3216 - accuracy: 0.8908 - val_loss: 0.3414 - val_accuracy: 0.8806
Epoch 5/5
53/53 [==============================] - 43s 805ms/step - loss: 0.3007 - accuracy: 0.8989 - val_loss: 0.3376 - val_accuracy: 0.8761


In [19]:
#Getting 89% training accuracy and 87% validation accuracy after FREEZING LAYERS

In [17]:
# More fine tuning the model
# Training the model after 140 layers
# Generally ResNet is good at extracting lower level features so we are not fine tuning initial layers
epochs = 10

split_at = 140                             #Freezing 140 layers and training the remaining 10 layers
for layer in model.layers[:split_at]: 
  layer.trainable = False
for layer in model.layers[split_at:]: 
  layer.trainable = True                  #Layer Training after 140 is True
    
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate, if we are fine tuning the initial layers
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

/Users/yashmishra12/miniforge3/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/var/folders/gw/057r0tcd6cg7wvd0kgbh499m0000gn/T/ipykernel_77395/3564746438.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10


2022-01-01 15:54:14.048524: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - ETA: 0s - loss: 0.3637 - accuracy: 0.8906

2022-01-01 15:55:01.905686: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - 58s 1s/step - loss: 0.3637 - accuracy: 0.8906 - val_loss: 0.2972 - val_accuracy: 0.8951
Epoch 2/10
53/53 [==============================] - 56s 1s/step - loss: 0.2420 - accuracy: 0.9300 - val_loss: 0.2628 - val_accuracy: 0.9085
Epoch 3/10
53/53 [==============================] - 57s 1s/step - loss: 0.1866 - accuracy: 0.9411 - val_loss: 0.2507 - val_accuracy: 0.9163
Epoch 4/10
53/53 [==============================] - 58s 1s/step - loss: 0.1487 - accuracy: 0.9560 - val_loss: 0.2495 - val_accuracy: 0.9118
Epoch 5/10
53/53 [==============================] - 57s 1s/step - loss: 0.1185 - accuracy: 0.9683 - val_loss: 0.2337 - val_accuracy: 0.9208
Epoch 6/10
53/53 [==============================] - 58s 1s/step - loss: 0.1029 - accuracy: 0.9705 - val_loss: 0.2309 - val_accuracy: 0.9297
Epoch 7/10
53/53 [==============================] - 56s 1s/step - loss: 0.0809 - accuracy: 0.9788 - val_loss: 0.2247 - val_accuracy: 0.9230
Epoch 8/10
53/53 [=============

## Observation

Improved validation accuracy to 93%

In [18]:
print('Training complete')

Training complete
